# 一、项目背景

## 1.1 案例研究

【案例研究】金融机构信用卡申请欺诈识别实践——基于"北X公寓"团伙攻击事件分析

在信用卡审批流程中，风险管理部门通过异常地址监测发现重大团伙欺诈线索：短期内集中涌现大量户籍地为A省B市的信用卡申请件，其邮寄地址高度统一指向"C市E区北三环xxx路xx号x楼北（晨/辰/宸）公寓"的特殊地址集群。经实地核验及交易回溯，确认该批申请系黑中介团伙以"免费办理POS机/收款二维码"为诱饵，非法收集客户身份信息后进行资质包装，通过代办信用卡申请并收取高额手续费实施欺诈的典型案件。

本批次案件呈现显著团伙作案特征：经系统筛查，邮寄地址字段包含"北晨/辰公寓"的申请件共计598件，涉及538名申请人。剔除节假日促销等特殊场景进件后，该地址日常申请量趋近于零，而本次攻击集中爆发于27-29日三天内，时间密度异常突出。申请人资质呈现系统性偏差——本科及以上学历占比仅3.18%，远低于正常客群水平；信用评分分布显著左偏，整体资质明显劣于常规进件。

值得关注的是，该团伙虽通过变换IP地址、手机号等基础伪装手段试图规避检测，但仍能通过多维关联规则引擎成功拦截多数风险申请。系统监测显示，涉案申请件的IP地址聚类系数、手机号关联度等指标呈现强相关性，触发"设备指纹异常""地址集群风险"等策略规则，形成有效防御屏障。

本案深刻揭示：欺诈与反欺诈是永不停歇的攻防博弈。当前黑产已形成"信息收集-资质伪造-批量申请-手续费分成"的完整产业链，攻击手段持续迭代升级。传统基于单一维度（如IP/手机号切换）的防御策略已难以应对新型团伙攻击，亟需构建多维智能识别体系。

此时，升级自然语言处理引擎，精准识别地址字段中的非常规字符变异（如"晨/辰/宸"同音字替换），建立地址热力图动态监测系统，对非常规地址集群实施秒级预警，强化风险识别能力，构建"数据驱动+智能算法+持续迭代"的立体化防控体系，方能在动态博弈中保持反欺诈防御优势。


## 1.2 任务说明

在收集示例数据之后，我们定义了以下地址匹配任务：

聚焦于地址文本的相关性评估：给定一条地址查询query与若干候选地址，模型需对二者的匹配程度进行打分。

* 输入：输入文件包含若干个query-地址文本对；
* 输出：输出文本每一行包括此query-地址文本对的匹配程度，分为完全匹配、部分匹配、不匹配；

示例：
```json
输入：
{
    "query": "江苏省南京市清水亭东路9号金域蓝湾15幢", 
    "candidate": [
        {
            "text": "江宁区万科金域蓝湾15栋"
        }, 
        {
            "text": "江苏省南京市清水亭东路9号"
        }, 
        {
            "text": "新水泥路666号重工数控工业园"
        }
    ]
}

输出：
{
    "query": "江苏省南京市清水亭东路9号金域蓝湾15幢", 
    "candidate": [
        {
            "text": "江宁区万科金域蓝湾15栋", 
            "label": "完全匹配"
        }, 
        {
            "text": "江苏省南京市清水亭东路9号", 
            "label": "部分匹配"
        }, 
        {
            "text": "新水泥路666号重工数控工业园", 
            "label": "不匹配"
        }
    ]
}
```

数据上，随机切分为训练集和测试集，可分情况选用训练集。

其中测试集的标签不可见，最终将在测试集上完成黑盒评测，以各个匹配程度标签F1平均值为目标分数。


## 1.3 传统算法

上述地址匹配任务面临的三大挑战如下：

* 写法多样：同一实体地址存在大量异构表达，且无官方改写词表可供查照。
* 语境约束：query 常携带省、市、区等限定信息，模型需据此精准判断候选地址的匹配与否。
* 规范差异：各地地址格式与习惯差异显著，对模型的跨域泛化能力提出了更高要求。

这些对于金融机构以地址库和规则匹配为核心的传统算法，有一定的难度。

以下，我们介绍传统算法其中之二（模型A、模型B），并在后续将它们作为准绳：


### 1.3.1 模型A

<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/ad32f2cfe905477db9e34fc15b5765b49b042189cf364e84810ed0bd78397ead" width="500">
</div>

基于某S数据服务厂商提供的地址匹配技术，需依赖内置的离线Quality Knowledge Base知识库K的Address模块，可理解为核心地址库+正则表达式库，再对地址进行标准化和Match code转换，转换后核心匹配逻辑类似于相似度计算。

### 1.3.2 模型B

<div align=center>
    <img src="https://ai-studio-static-online.cdn.bcebos.com/54f5b51bc7a5479d88eeff4e00937e19b29c463ba71c4aaf85264a33cbf33225" width="500">
</div>

某X金融科技自研的工业化算法，该模型基于模型A开发，进一步强化标准地址库，复合更细的专家匹配规则，融合多维度相似度，采用动态更新和增量训练机制，持续提升模型泛化能力。

### 1.3.3 模型C

这是一个简单的模型融合，用于模型对比，对于模型A、模型B各取所长。


# 二、文心大模型

特别说明，由于传统算法有比较复杂的地址库和规则引擎，不一定具备直接进行模型细调/Fine-Tuning的能力。

我们在此区分两种场景进行对比：

**(1) 泛化能力**

仅在测试集上，调用模型A、模型B、文心ERNIE系列模型，直接进行黑盒评测，对比其F1分数。

**(2) 模型细调**

引入训练集，基于文心ERNIE系列模型进行SFT细调，随后在测试集上进行黑盒评测，对比其F1分数。


## 2.1 泛化能力

### 2.1.1 快速部署

一键加载，自动下载，速度400M/s+，优秀到起飞！

注意：需要Tesla A800 80GB，实际显存占用50%～95%

* 0.server.sh

```shell
pip install pandarallel

clear
path=/home/aistudio/data/models
model=ERNIE-4.5-21B-A3B-Base-Paddle

echo $model
rm -rf $path/$model
aistudio download --model PaddlePaddle/$model --local_dir $path/$model
ls -l $path/$model

python -m fastdeploy.entrypoints.openai.api_server --model $path/$model --port 8180 --metrics-port 8181 --engine-worker-queue-port 8182 --max-model-len 1024 --max-num-seqs 128
```


### 2.1.2 Prompt工程

这里推荐**模型体验场**[https://aistudio.baidu.com/playground](https://aistudio.baidu.com/playground)

![](https://ai-studio-static-online.cdn.bcebos.com/8ae8fb1392d84633979b6fa34aa118580e33f68f916d4911969eb930067d7ab9)

可进行模型配置调试后，生成并复制相关代码。

* 1.test.py

```python
import os
import sys
import json
import openai
import pandas as pd

# Import
from pandarallel import pandarallel
# Initialization
pandarallel.initialize(nb_workers=64, progress_bar=True)

try:
    work_txt = sys.argv[1]
except:
    work_txt = "x3nlp_1_test.txt"
# os.system("clear")
print(f"work_txt: {work_txt}")

host = "0.0.0.0"
port = "8180"
client = openai.Client(base_url=f"http://{host}:{port}/v1", api_key="null")

_system = {
    "role": "system", 
    "content": """
你是一个地址匹配专家，你需要判断输入的json中，query和candidate下的每一个text都是否为同一个地址信息，如果匹配正确则label为"完全匹配"，如果匹配不正确则label为"不匹配"，如果只有部分匹配则label为"部分匹配"，返回结果严格按照json格式。

输入：
{"query": "江苏省南京市清水亭东路9号金域蓝湾15幢", "candidate": [{"text": "江宁区万科金域蓝湾15栋"}, {"text": "江苏省南京市清水亭东路9号"}, {"text": "新水泥路666号重工数控工业园"}]}

输出：
{"query": "江苏省南京市清水亭东路9号金域蓝湾15幢", "candidate": [{"text": "江宁区万科金域蓝湾15栋", "label": "完全匹配"}, {"text": "江苏省南京市清水亭东路9号", "label": "部分匹配"}, {"text": "新水泥路666号重工数控工业园", "label": "不匹配"}]}
"""
}


def get(_c, _i=""):
    _jc = json.loads(_c)
    _jc_id = _jc["text_id"]
    _c = json.dumps({
        "query": _jc["query"],
        "candidate": [{"text": i["text"]} for i in _jc["candidate"]],
    }, ensure_ascii=False)
    # print(_c)

    response = client.chat.completions.create(
        model="null",
        messages=[_system, {"role": "user", "content": _c}],
        stream=False,

        extra_body={
            "penalty_score": 0.0,   # 关闭惩罚，避免干扰匹配
        },
        max_completion_tokens=512,  # 地址通常较短，
        temperature=0.01,           # 完全确定性输出，避免随机性
        top_p=1.0,                  # 关闭核采样（与temperature=0冲突，可保留）
        frequency_penalty=0.0,      # 不惩罚重复（地址可能有重复元素）
        presence_penalty=0.0,       # 不惩罚存在性（关键信息需保留）

        response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "candidate": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "text": {"type": "string"},
                                "label": {"type": "string", "enum": ["完全匹配", "部分匹配", "不匹配"]}
                            },
                            "required": ["text", "label"]
                        }
                    }
                },
                "required": ["query", "candidate"]
            }
        }
    )

    try:
        _j = json.loads(
            response.choices[0].message.content.replace("`","").replace("json","")
        )
    except Exception as e:
        # print(f"\n \033[1;36m ERROR {_i}:\033[0m \n{response.choices[0].message.content[:100]}\n{e}", )
        _j = json.loads(_c)
    finally:
        _j["text_id"] = _jc_id

        for _ji in _j["candidate"]:
            if "label" not in _ji:
                _ji.update({"label": "?匹配?"})
        return json.dumps(_j, ensure_ascii=False)


t = """
{"text_id": "2b51366fdd6c620a3f54b520a8ebc5e5", "query": "兴东街道铁成佳园农贸大市场2厅", "candidate": [{"text": "沙河镇街9-1号铁成佳园农贸大市场", "label": "完全匹配"}, {"text": "沙河镇街11号铁成佳园农贸大市场(东南1门)", "label": "不匹配"}, {"text": "宏安路与沙河镇街交叉口北50米铁成佳园农贸大市场(东南2门)", "label": "部分匹配"}, {"text": "37县道佳园农场", "label": "部分匹配"}, {"text": "杭州大道农贸大市场", "label": "不匹配"}]}
"""
print(f"Test:\n{t}\n{get(t)}\nStart:\n")
# raise "Test"

data = pd.read_csv(
    work_txt, sep="\t", header=None, names=["text"]
)
# parallel_apply
data["result"] = data["text"].parallel_apply(get)

with open("test.txt", "w") as f:
    for i in data["result"]:
        f.write(f"{i}\n")

```

其中，模型配置上，考虑地址匹配场景，使用了以下设置：


In [ ]:
extra_body={
    "penalty_score": 0.0,   # 关闭惩罚，避免干扰匹配
},
max_completion_tokens=512,  # 地址通常较短，
temperature=0.01,           # 完全确定性输出，避免随机性
top_p=1.0,                  # 关闭核采样（与temperature=0冲突，可保留）
frequency_penalty=0.0,      # 不惩罚重复（地址可能有重复元素）
presence_penalty=0.0,       # 不惩罚存在性（关键信息需保留）

返回的json问题，也通过response_format定义清楚，这要比在content中限制更为明确一些：

In [ ]:
response_format={
    "type": "json_object",
    "schema": {
        "type": "object",
        "properties": {
            "query": {"type": "string"},
            "candidate": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "text": {"type": "string"},
                        "label": {"type": "string", "enum": ["完全匹配", "部分匹配", "不匹配"]}
                    },
                    "required": ["text", "label"]
                }
            }
        },
        "required": ["query", "candidate"]
    }
}

特别的，为了响应并发调用，这里使用了parallel_apply调用，效果还可以，仅供参考。

In [ ]:

# Import
from pandarallel import pandarallel
# Initialization
pandarallel.initialize(nb_workers=64, progress_bar=True)

data = pd.read_csv(
    work_txt, sep="\t", header=None, names=["text"]
)
# parallel_apply
data["result"] = data["text"].parallel_apply(get)


### 2.1.3 格式优化

由于json输出是我们的本质要求，也进行了特别说明和提示词定义，但为保证评测准确，我们对输出结果进行格式化：

* 2.format.py

```python
import os
import sys
import json
from tqdm import tqdm

try:
    work_txt = sys.argv[1]
except:
    work_txt = "x3nlp_1_test.txt"
# os.system("clear")
print(f"work_txt: {work_txt}")

# 
# 

def lookup(k, l, _k):
    for i in l:
        if i["text"] == k:
            return {
                "完全匹配": "完全匹配",
                "部分匹配": "部分匹配",
                "不匹配": "不匹配",

                "?匹配?": "不匹配",
            }.get(i["label"], "不匹配")
    print("X1")
    return "部分匹配"

w = open("test_format.txt", "w")
with open(work_txt, "r") as f:
    for j0 in tqdm(f, total=15000):
        j0 = json.loads(j0)

        try:
            j1 = os.popen(f"cat test.txt|grep {j0['text_id']}").readlines()[0]
        except:
            j1 = '{"candidate": []}'
            print("X2")
        j1 = json.loads(j1)

        # format
        j2 = {
            "text_id": j0["text_id"],
            "query": j0["query"],
            "candidate": [
                {
                    "text": i["text"],
                    "label": lookup(i["text"], j1["candidate"], j0["query"]),
                }
                for i in j0["candidate"]
            ]
        }
        w.write(f"{json.dumps(j2, ensure_ascii=False)}\n")
        # print(f"{'-'*100}\nj0: {j0}\nj1: {j1}\nj2: {j2}\n{'-'*100}")
        # break
w.close()
```

将一些输出故障，暂时性缺省为不匹配。

### 2.1.4 输出检查

* 3.check.py

```python
import json
import linecache


def check(submit_path, test_path, max_num=15000):
    '''
    :param submit_path: 提交的文件名
    :param test_path: 原始测试数据名
    :param max_num: 测试数据大小
    :return:
    '''
    N = 0
    with open(submit_path, 'r', encoding='utf-8') as fs:
        for line in fs:
            try:
                line = line.strip()
                if line == '':
                    continue
                N += 1
                smt_jsl = json.loads(line)
                test_jsl = json.loads(linecache.getline(test_path, N))
                if set(smt_jsl.keys()) != {'text_id', 'query', 'candidate'}:
                    raise AssertionError(f'请保证提交的JSON数据的所有key与测评数据一致！ {line}')
                elif smt_jsl['text_id'] != test_jsl['text_id']:
                    raise AssertionError(f'请保证text_id和测评数据一致，并且不要改变数据顺序！text_id: {smt_jsl["text_id"]}')
                elif smt_jsl['query'] != test_jsl['query']:
                    raise AssertionError(f'请保证query内容和测评数据一致！text_id: {smt_jsl["text_id"]}, query: {smt_jsl["query"]}')
                elif len(smt_jsl['candidate']) != len(test_jsl['candidate']):
                    raise AssertionError(f'请保证candidate的数量和测评数据一致！text_id: {smt_jsl["text_id"]}')
                else:
                    for smt_cand, test_cand in zip(smt_jsl['candidate'], test_jsl['candidate']):
                        if smt_cand['text'] != test_cand['text']:
                            raise AssertionError(f'请保证candidate的内容和顺序与测评数据一致！text_id: {smt_jsl["text_id"]}, candidate_item: {smt_cand["text"]}')

            except json.decoder.JSONDecodeError:
                raise AssertionError(f'请检查提交数据的JSON格式是否有误！如：用键-值用双引号 {line}')
            except KeyError:
                raise AssertionError(f'请保证提交的JSON数据的所有key与测评数据一致！{line}')

    if N != max_num:
        raise AssertionError(f"请保证测试数据的完整性(共{max_num}条)，不可丢失或增加数据！")

    print('Well Done ！！')

check('test_format.txt', 'x3nlp_1_test.txt')
```
保证数据条目、格式等等均正常，输出Well Done ！！即可！



## 2.2 模型细调

参考 [https://github.com/PaddlePaddle/ERNIE/blob/release/v1.5/docs/erniekit.md](https://github.com/PaddlePaddle/ERNIE/blob/release/v1.5/docs/erniekit.md)

### 2.2.1 数据准备

参考 [https://github.com/PaddlePaddle/ERNIE/tree/release/v1.5/examples/data](https://github.com/PaddlePaddle/ERNIE/tree/release/v1.5/examples/data)

这里我们的SFT Supervised Fine-tuning 采用的数据格式如下：

```shell
# https://github.com/PaddlePaddle/ERNIE/blob/release/v1.5/examples/data/sft-eval.jsonl
{"src": ["列出三首在90年代流行的歌曲。"], "tgt": ["\"I Want It That Way\"（唱：Backstreet Boys）、\"Torn\"（唱：Natalie Imbruglia）和\"Livin' La Vida Loca\"（唱：Ricky Martin）。"]}
{"src": ["描述正确处理危险废物的方法。"], "tgt": ["危险废物应该被收集在密封容器中，并带到授权的危险废物处理设施。容器应标有内部废物的类型和其处理方法的说明。"]}
{"src": ["写一个关于时间旅行者的创意故事。"], "tgt": ["从前，有一个叫蒂姆的年轻人，他热爱探索和体验新的地方和文化。他偶然得到了一只神奇的手表，可以让他回到过去。他利用这只手表穿越不同的时期，去探索他祖先生活过的世界。他的冒险带他穿越野西到古老的朝代，来回穿梭于时空之间，试图解决几个世纪以来无法解决的难题和谜题。"]}
{"src": ["提供三个使用“redundant”一词的例句"], "tgt": ["1. 这份文件包含了很多不必要的冗余信息。\n2. 系统更新后，一些冗余代码被删除了。\n3. 有冗余的备份保证了数据不会永久丢失。"]}
{"src": ["辨别以下短语是真是假：“宇宙正在不断扩张。”"], "tgt": ["真。"]}
{"src": ["从短语“人工智能教育”中创建三个研究论文标题"], "tgt": ["1. 人工智能教育有效性的全面研究\n2. 利用人工智能在教育中面临的挑战和机遇\n3. 探索将人工智能作为增强教育成果的工具"]}
{"src": ["给我一本受欢迎的儿童书的名称。"], "tgt": ["猫和帽子里的猫。"]}
{"src": ["在南美洲中找出三个国家"], "tgt": ["巴西、哥伦比亚和阿根廷。"]}
{"src": ["将华氏温度80度转换为相应的摄氏温度"], "tgt": ["26.7摄氏度。"]}
{"src": ["写三句话描述一个宁静的海滩"], "tgt": ["1. 当柔和的海浪缓缓地涌入海滩时，太阳在清澈的蓝色海洋上闪闪发光。\n2.沙滩上的白沙在轻微的微风中闪闪发光。\n3.一群海鸥在海浪的伴随中高歌，并齐声吟唱。"]}
```
根据这个格式，准备数据：


In [ ]:
for _name, _data in [
    ["SFT-train.jsonl", d1],
    ["SFT-eval.jsonl", d2],
]:
    with open(_name, "w") as f:
        for i in _data["text"]:
            j = json.loads(i)
            j1 = {
                "query": j["query"], 
                "candidate": [{"text": ij["text"]} for ij in j["candidate"]]
            }
            j2 = {
                "query": j["query"], 
                "candidate": j["candidate"]
            }
            j3 = {
                "src": [json.dumps(j1, ensure_ascii=False)],
                "tgt": [json.dumps(j2, ensure_ascii=False)],
            }
            # print(j1, "\n\n", j2, "\n\n", j3)
            f.write(json.dumps(j3, ensure_ascii=False)+"\n")
            # break

### 2.2.2 开始训练

* 参数配置/SFT-run_sft_lora_8k.yaml

```yaml
### data
train_dataset_type: "erniekit"
eval_dataset_type: "erniekit"
train_dataset_path: "./SFT-train.jsonl"
train_dataset_prob: "1.0"
eval_dataset_path: "./SFT-eval.jsonl"
eval_dataset_prob: "1.0"
max_seq_len: 8192
num_samples_each_epoch: 6000000

### model
model_name_or_path: data/models/ERNIE-4.5-21B-A3B-Base-Paddle
moe_group: mp
fine_tuning: LoRA
lora_rank: 32
fuse_rope: True
use_sparse_head_and_loss_fn: True

### finetuning
# base
stage: SFT
seed: 23
do_train: True
do_eval: True
distributed_dataloader: False
dataloader_num_workers: 1
batch_size: 1
num_train_epochs: 1
max_steps: 600
max_evaluate_steps: 10000
eval_steps: 10000
evaluation_strategy: steps
save_steps: 10000000
save_total_limit: 5
save_strategy: steps
logging_steps: 1
release_grads: True
gradient_accumulation_steps: 8
logging_dir: ./vdl_log
output_dir: data/models/ERNIE-4.5-21B-A3B-Base-LORA
disable_tqdm: True

# train
warmup_steps: 20
learning_rate: 3.0e-4
lr_scheduler_type: cosine
min_lr: 1.0e-6
layerwise_lr_decay_bound: 1.0

# optimizer
weight_decay: 0.1
adam_epsilon: 1.0e-8
adam_beta1: 0.9
adam_beta2: 0.95
offload_optim: True

# performance
tensor_parallel_degree: 1
pipeline_parallel_degree: 1
sharding_parallel_degree: 1
sharding: stage1
sequence_parallel: True
pipeline_parallel_config: disable_partial_send_recv enable_clear_every_step_cache
recompute: True
recompute_use_reentrant: True
compute_type: bf16
fp16_opt_level: O2
disable_ckpt_quant: True
amp_master_grad: True
amp_custom_white_list:
  - lookup_table
  - lookup_table_v2
  - flash_attn
  - matmul
  - matmul_v2
  - fused_gemm_epilogue
amp_custom_black_list:
  - reduce_sum
  - softmax_with_cross_entropy
  - c_softmax_with_cross_entropy
  - elementwise_div
  - sin
  - cos
unified_checkpoint: True
unified_checkpoint_config: async_save
```

主要配置：

* SFT训练

train_dataset_path: "./SFT-train.jsonl"

* SFT验证

eval_dataset_path: "./SFT-eval.jsonl"

* 模型路径

model_name_or_path: data/models/ERNIE-4.5-21B-A3B-Base-Paddle

* 运行步数

max_steps: 600

* 输出路径

output_dir: data/models/ERNIE-4.5-21B-A3B-Base-LORA

然后，开始SFT：

```python
os.system(f"rm -rf {p1}")
os.system("erniekit train SFT-run_sft_lora_8k.yaml")
```

### 2.2.3 合并权重

* 参数配置/SFT-run_export.yaml

```yaml
### model
model_name_or_path: data/models/ERNIE-4.5-21B-A3B-Base-Paddle
fine_tuning: LoRA

### split
max_shard_size: 5
hf_hub_id: null
output_dir: data/models/ERNIE-4.5-21B-A3B-Base-LORA

### performance
tensor_parallel_degree: 1
pipeline_parallel_degree: 1
sharding_parallel_degree: 1
sharding: stage1
pipeline_parallel_config: disable_partial_send_recv enable_clear_every_step_cache
sequence_parallel: True
compute_type: bf16
fp16_opt_level: O2
```

主要配置：

* 模型路径
model_name_or_path: data/models/ERNIE-4.5-21B-A3B-Base-Paddle

* 输出路径
output_dir: data/models/ERNIE-4.5-21B-A3B-Base-LORA

然后，开始合并：

```python
os.system(f"rm -rf {p1}/export")
os.system("erniekit export SFT-run_export.yaml lora=True")
```


### 2.2.4 重新部署

参考 2.1.1 快速部署，修改模型路径。

```python
os.system(f"python -m fastdeploy.entrypoints.openai.api_server --model {p1}/export --port 8180 --metrics-port 8181 --engine-worker-queue-port 8182 --max-model-len 1024 --max-num-seqs 128")
```


* 4.SFT.py

```python
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split

os.system("clear")


# 1.Data Preparation
# CUT TRAIN && EVAL DATA TO JSONL
data = pd.read_csv(
    "x3nlp_1_train.txt", sep="\t", header=None, names=["text"]
)
d1, d2 = train_test_split(data, test_size=0.2)
print(d1.shape, d2.shape)

for _name, _data in [
    ["SFT-train.jsonl", d1],
    ["SFT-eval.jsonl", d2],
]:
    with open(_name, "w") as f:
        for i in _data["text"]:
            j = json.loads(i)
            j1 = {
                "query": j["query"], 
                "candidate": [{"text": ij["text"]} for ij in j["candidate"]]
            }
            j2 = {
                "query": j["query"], 
                "candidate": j["candidate"]
            }
            j3 = {
                "src": [json.dumps(j1, ensure_ascii=False)],
                "tgt": [json.dumps(j2, ensure_ascii=False)],
            }
            # print(j1, "\n\n", j2, "\n\n", j3)
            f.write(json.dumps(j3, ensure_ascii=False)+"\n")
            # break


p1 = "data/models/ERNIE-4.5-21B-A3B-Base-LORA"

# 2.Supervised Fine-tuning
os.system(f"rm -rf {p1}")
os.system("erniekit train SFT-run_sft_lora_8k.yaml")


# 3.Weight Merging
os.system(f"rm -rf {p1}/export")
os.system("erniekit export SFT-run_export.yaml lora=True")


# 4.Load Model To Server
os.system(f"python -m fastdeploy.entrypoints.openai.api_server --model {p1}/export --port 8180 --metrics-port 8181 --engine-worker-queue-port 8182 --max-model-len 1024 --max-num-seqs 128")


```


# 三、结果对比

## 3.1 泛化能力

### 3.1.1 模型A

|Model|F1|Note|
|-|-|-|
|模型A|**0.2653**||

模型A简单直接，但从F1分数上看，能力有点一般。

### 3.1.2 模型B

|Model|F1|Note|
|-|-|-|
|模型B|**0.3483**||

事实上，确实模型B要比模型A更好，F1分数有0.08+提升。

### 3.1.3 模型C

|Model|F1|Note|
|-|-|-|
|模型C|**0.3696**|CROSS(A+B)|

模型融合后的模型C，F1分数有0.02提升，融合模型A后稍有增益。

### 3.1.4 ERNIE-4.5-0.3B-Base

|Model|F1|Note|
|-|-|-|
|ERNIE-4.5-0.3B-Base|**0.3719**|*para|

参数规模0.3B，基础预训练模型，极致轻量，推理速度快，比模型B、模型C稍好，能力相对持平。

### 3.1.5 ERNIE-4.5-21B-A3B

|Model|F1|Note|
|-|-|-|
|ERNIE-4.5-21B-A3B|**0.4843**|*para|

参数规模21B，中大型任务，科研实验常用，F1分数比模型C有0.11+提升，开始降维打击！

### 3.1.6 ERNIE-4.5-21B-A3B-Base

|Model|F1|Note|
|-|-|-|
|ERNIE-4.5-21B-A3B-Base|**0.5537**|*para|

参数规模21B，擅长通用文本生成，中型任务，F1分数比模型C有0.18+提升，直接拉开差距！强！

## 3.2 模型细调

### 3.2.1 ERNIE-4.5-21B-A3B-Base-LORA

|Model|F1|Note|
|-|-|-|
|ERNIE-4.5-21B-A3B-Base|**0.7741**|*para && SFT-lora-8k-2w-S600-.8|

基于相同的ERNIE-4.5-21B-A3B-Base，切取80%训练集作为SFT LORA 8K的模型细调，经过600steps之后，F1分数再有0.22+提升，进一步拉开差距！当然，我们也补充了其他形式的NLP模型进行训练，F1分数大概在0.52-0.82，细调后已具中上水平了。

当然，更加大的参数规模和更加深刻的SFT，效果可能还会更好！这就留作后话吧！有兴趣的朋友们可以自行试试！

特别说明，*para 即同一参数设置，详见章节二。


# 四、策略展望

随着自然语言处理引擎的升级，对上述反欺诈策略在不同业务阶段进行流程复盘，并推演策略全景。

## 4.1 初级阶段

![](https://ai-studio-static-online.cdn.bcebos.com/cb7d800606d440969200dff95c5480b9c3d76367e42444e0a606c74f46d6721a)

主要依靠纯人工审批，或传统黑名单、专家经验为主的业务规则。当发现疑似个案后，通过各系统简单查询，发现由疑似个案牵连出来的欺诈群体，并根据人工判断是否实际欺诈，结合不同申请状态采用对应的反制举措。重度依赖审批专家经验判别，准确性较好，但人工成本过高，且案件排查未能充分挖掘出与疑似个案关联所有申请件。

## 4.2 策略进阶

![](https://ai-studio-static-online.cdn.bcebos.com/7cf62a0a24fe4505be6c962880117d2124e02b0f9c01425689f632b52779c1d9)

地址匹配能力升级之后，类似于**模型A、模型B、模型C、ERNIE-4.5-0.3B-Base**。

在初级阶段基础上，先后引入反欺诈模型、关联网络技术，强化数据验证、数据反馈机制，建立黑名单库。在关联网络+模糊匹配加持下，由欺诈个案将牵连出更完整的欺诈社团，并更具模型评分判断是否实际欺诈。反欺诈模型坏样本少，需通过贷前审批、贷后催收反馈收集，模型更易学会已知欺诈模式，而对未知模式可能束手无策，需要快速迭代。

## 4.3 全景推演

![](https://ai-studio-static-online.cdn.bcebos.com/077e76aa2c6a4b439d766f68f8f4c3e5b0eae43b32b84734a22c6fe06fd61a9b)

如果能有更强的匹配技术，如：**ERNIE-4.5-21B-A3B、ERNIE-4.5-21B-A3B-Base、ERNIE-4.5-21B-A3B-Base-LORA**。

据此推演更加全面的反欺诈策略流程：

● 强化信息验证

引入外部数据、埋点数据，并充分考虑查询成本、客户体验；

● 获取潜在关系

实时计算新进申请件所在关联网络结果，返回其欺诈网络、关联特征等；

● 完善数据反馈

打通贷前审批、案件调查、舆情监控、贷中异常、贷后催收等业务环节的数据反馈收集；

● 识别欺诈模式

不断数据积累，据此构建覆盖已知+未知欺诈模式的反欺诈模型，并返回欺诈评分，以及对应群体的电核问题、业务提示，降低电核成本；


# 写在最后

关于智能金融方向，有兴趣的朋友们可以查阅[智能金融之决策科学篇](https://aistudio.baidu.com/projectdetail/8623759)，希望可以帮到大家。

更多的，欢迎关注[AI Studio](https://aistudio.baidu.com/personalcenter/thirdview/979775)/[GitHub](https://github.com/IvanaXu)。
